In [1]:
#!pip install h2o
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
from benchmark_utils import timer


In [2]:
h2o.init()
X_train = h2o.import_file('../datasets/porto-seguro/train.csv',header=1)
X_test = h2o.import_file('../datasets/porto-seguro/test.csv',header=1)

id_test = X_test["id"].as_

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_152-release"; OpenJDK Runtime Environment (build 1.8.0_152-release-1056-b12); OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)
  Starting server from /home/midas/anaconda3/envs/automl-3.7/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp871g6j8i
  JVM stdout: /tmp/tmp871g6j8i/h2o_midas_started_from_python.out
  JVM stderr: /tmp/tmp871g6j8i/h2o_midas_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.5
H2O cluster version age:,1 month and 15 days
H2O cluster name:,H2O_from_python_midas_vwmyeg
H2O cluster total nodes:,1
H2O cluster free memory:,26.67 Gb
H2O cluster total cores:,20
H2O cluster allowed cores:,20
H2O cluster status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
X_train.head()

id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
7,0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.71807,10,1,-1,0,1,4,1,0,0,1,12,2,0.4,0.883679,0.37081,3.60555,0.6,0.5,0.2,3,1,10,1,10,1,5,9,1,5,8,0,1,1,0,0,1
9,0,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316228,0.618817,0.388716,2.44949,0.3,0.1,0.3,2,1,9,5,8,1,7,3,1,1,9,0,1,1,0,1,0
13,0,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0,0,-1,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316228,0.641586,0.347275,3.31662,0.5,0.7,0.1,2,2,9,1,8,2,7,4,2,7,7,0,1,1,0,1,0
16,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,7,1,0,0,1,11,1,1,3,1,104,1,0.374166,0.542949,0.294958,2,0.6,0.9,0.1,2,4,7,1,8,4,2,2,2,4,9,0,0,0,0,0,0
17,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840759,11,1,-1,0,-1,14,1,1,2,1,82,3,0.31607,0.565832,0.365103,2,0.4,0.6,0,2,2,6,3,10,2,12,3,1,1,3,0,0,0,1,1,0
19,0,5,1,4,0,0,0,0,0,1,0,0,0,0,0,6,1,0,0,0.9,1.8,2.33265,10,0,-1,0,0,14,1,1,0,1,104,2,0.445982,0.879049,0.406202,3,0.7,0.8,0.4,3,1,8,2,11,3,8,4,2,0,9,0,1,0,1,1,1
20,0,2,1,3,1,0,0,1,0,0,0,0,0,0,0,8,1,0,0,0.6,0.1,0.617454,6,1,-1,0,1,11,1,1,0,1,99,2,0.316228,0.639683,0.368782,3.16228,0.2,0.6,0.5,2,2,8,1,8,3,10,3,0,0,10,0,1,0,0,1,0
22,0,5,1,4,0,0,1,0,0,0,0,0,0,0,0,13,1,0,0,0.7,0.4,0.607248,11,1,0,0,0,11,1,1,2,1,30,3,0.44699,0.900574,0.374833,3.31662,0.1,0.5,0.1,1,2,7,1,6,1,13,7,1,3,6,1,0,1,0,1,0
26,0,5,1,3,1,0,0,0,1,0,0,0,0,0,0,6,1,0,0,0.9,0.7,0.901388,10,1,-1,0,1,14,1,1,0,1,68,3,0.4,0.780641,0.407431,2.82843,0.9,0.8,0.6,3,1,7,3,9,4,11,4,2,1,5,0,1,0,0,0,1
28,1,1,1,2,0,0,0,1,0,0,0,0,0,0,0,4,0,0,1,0.9,1.4,2.31665,11,0,0,1,0,14,1,1,2,1,104,2,0.447214,1.45818,0.390256,3.60555,0.7,0.8,0.8,2,2,8,2,9,1,11,3,5,0,6,0,1,0,0,1,0


## Treinamento

In [4]:
start_time = timer(None)

x = X_train.columns
y = "target"

X_train[y] = X_train[y].asfactor()

# (limited to 1 hour max runtime by default)
aml = H2OAutoML()
aml.train(x=x, y=y, training_frame=X_train)
time = timer(start_time)



AutoML progress: |████████████████████████████████████████████████████████| 100%


In [5]:
print(time)



 Time taken: 1 hours 0 minutes and 36.85 seconds.


In [6]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)


model_id,auc,logloss,mean_per_class_error,rmse,mse
XGBoost_3_AutoML_20191101_013359,0.638652,0.152015,0.431207,0.186416,0.0347508
StackedEnsemble_AllModels_AutoML_20191101_013359,0.636464,0.152637,0.435221,0.186652,0.0348391
StackedEnsemble_BestOfFamily_AutoML_20191101_013359,0.636287,0.152697,0.435457,0.186661,0.0348423
GBM_1_AutoML_20191101_013359,0.636164,0.152554,0.439062,0.18681,0.0348981
GBM_2_AutoML_20191101_013359,0.634262,0.15255,0.434575,0.186658,0.0348413
GBM_3_AutoML_20191101_013359,0.63359,0.152735,0.432122,0.186783,0.0348879
XGBoost_1_AutoML_20191101_013359,0.63282,0.152427,0.437469,0.186543,0.0347984
XGBoost_2_AutoML_20191101_013359,0.624014,0.153109,0.437014,0.186692,0.0348539
GLM_grid_1_AutoML_20191101_013359_model_1,0.621661,0.15315,0.441903,0.186727,0.0348668
GBM_4_AutoML_20191101_013359,0.621606,0.15387,0.438074,0.187207,0.0350464


In [7]:
# The leader model is stored here
aml.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_3_AutoML_20191101_013359


Model Summary: 

,,number_of_trees
0,,154.0




ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.03452921779447898
RMSE: 0.1858203912235656
LogLoss: 0.1500868172935473
Mean Per-Class Error: 0.3842044648466768
AUC: 0.6637642029105052
pr_auc: 0.0832912768906451
Gini: 0.3275284058210104

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.059872064558702684: 

,,0,1,Error,Rate
0,0,525521.0,47997.0,0.0837,(47997.0/573518.0)
1,1,16599.0,5095.0,0.7651,(16599.0/21694.0)
2,Total,542120.0,53092.0,0.1085,(64596.0/595212.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


Scoring History: 

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2019-11-01 01:53:53,2 min 53.332 sec,0.0,0.500000,0.693147,0.500000,0.000000,1.000000,0.963552
1,,2019-11-01 01:53:55,2 min 54.798 sec,5.0,0.404936,0.518678,0.625367,0.065699,3.711023,0.148582
2,,2019-11-01 01:53:55,2 min 55.637 sec,10.0,0.335537,0.406338,0.628924,0.067239,3.760853,0.113081
3,,2019-11-01 01:53:56,2 min 56.458 sec,15.0,0.285862,0.330315,0.629603,0.067795,3.892549,0.123892
4,,2019-11-01 01:53:57,2 min 57.320 sec,20.0,0.251228,0.277454,0.630781,0.068162,3.857639,0.120372
5,,2019-11-01 01:53:58,2 min 58.176 sec,25.0,0.227793,0.240185,0.630063,0.068614,3.857639,0.113371
6,,2019-11-01 01:53:59,2 min 59.058 sec,30.0,0.212385,0.213748,0.630148,0.069061,3.935990,0.114005
7,,2019-11-01 01:54:00,2 min 59.981 sec,35.0,0.202510,0.195003,0.631285,0.069394,3.903073,0.107972
8,,2019-11-01 01:54:01,3 min 0.898 sec,40.0,0.196300,0.181743,0.631877,0.069598,3.948491,0.113884
9,,2019-11-01 01:54:02,3 min 1.805 sec,45.0,0.192451,0.172440,0.633219,0.069800,3.922164,0.105562



See the whole table with table.as_data_frame()

Variable Importances: 

,variable,relative_importance,scaled_importance,percentage
0,ps_car_13,11442.995117,1.000000,0.177085
1,ps_ind_05_cat,5636.885742,0.492606,0.087233
2,ps_reg_03,5006.640625,0.437529,0.077480
3,ps_ind_03,4832.136719,0.422279,0.074779
4,ps_ind_17_bin,3725.914062,0.325607,0.057660
5,ps_ind_15,2863.300049,0.250223,0.044311
6,ps_car_07_cat,2215.993652,0.193655,0.034293
7,ps_reg_02,2179.951660,0.190505,0.033736
8,ps_car_03_cat,1927.481323,0.168442,0.029828
9,ps_car_01_cat,1905.303711,0.166504,0.029485



See the whole table with table.as_data_frame()


In [39]:
preds = aml.leader.predict(X_test)
print(preds)

xgboost prediction progress: |████████████████████████████████████████████| 100%


predict,p0,p1
0,0.971546,0.0284544
0,0.97192,0.0280801
0,0.96991,0.0300901
0,0.983864,0.0161361
0,0.961902,0.0380978
0,0.960884,0.0391159
0,0.980113,0.0198871
0,0.964027,0.0359726
0,0.943828,0.0561724
0,0.94634,0.0536596


In [40]:
h2o.save_model(aml.leader, path = "Porto_seguro_h2o")

H2OResponseError: Server error water.exceptions.H2OIllegalArgumentException:
  Error: Illegal argument: dir of function: exportModel: water.api.FSIOException: FS IO Failure: 
 accessed path : file:/home/midas/disk2/sette/git/automl-notebooks-benchmark/porto_seguro/Porto_seguro_h2o/XGBoost_3_AutoML_20191101_013359 msg: File already exists
  Request: GET /99/Models.bin/XGBoost_3_AutoML_20191101_013359
    params: {'dir': 'Porto_seguro_h2o/XGBoost_3_AutoML_20191101_013359', 'force': 'False'}


In [41]:
print(time)
preds = preds.as_data_frame().predict.values
id_test = X_test.as_data_frame().id.values


 Time taken: 1 hours 0 minutes and 36.85 seconds.


In [43]:
submission = pd.DataFrame({
        "id": id_test,
        "target": preds
    })

submission.to_csv("../submit_files/porto_seguro_h2o_submission.csv", index=False)